## TauRunner

Welcome to the TauRunner tutorial! This software is capable of simulating neutrinos (and anti-neutrinos) of all flavors, accounting for tau regeneration and losses in the nutau channel. The output is a set of particle energies, number of interactions, angle, and particle type.

The user has the option to propagate:
1. monochromatic fluxes
2. power-law fluxes
3. Any provided arbitrary flux by means of sampling from a cdf. 

All of these can be propagated through either:
1. a fixed angle
2. a range of angles
3. uniformly sampled angles in the sky. 

In terms of astrophysical bodies, we provide implementations for the Earth and Sun with capabilities of adding additional layers and changing the detector depth. In addition, we will demonstrate how to define any astrophysical object, or a slab of constant density to propagate through. All of these capabilities will be shown in the following examples. Happy running!

## Example 1: Monochromatic flux through single angle in Earth ##

The main function that runs the monte carlo is called `run_MC` and required a few arguments, which we show you how to create below
* `energies`: An array of initial particle energies in eV
* `thetas`: A corresponding array of initial nadir angles in radians
* `body`: The body which we are propagating through (Earth, Sun, custom, etc.)
* `xs`: a CrossSections object which is based on a certain cross section model
* `propagator`: PROPOSAL propagator object for charged lepton propagation

In [ ]:
import numpy as np

from taurunner.main import run_MC
from taurunner.body.earth import construct_earth
from taurunner.cross_sections import CrossSections
from taurunner.utils import make_initial_e, make_initial_thetas

nevents  = 5000 # number of events to simulate
eini = 1e19 # initial energy in eV
theta = 89.0 # incidence angle (nadir)
pid = 16 # PDG MC Encoding particle ID (nutau)
xs_model = "CSMS" # neutrino cross section model
random_seed = 925

earth = construct_earth(layers=[(4., 1.0)]) # Make Earth object with 4km water layer
xs = CrossSections(xs_model)
energies = make_initial_e(nevents, eini) # Return array of initial energies in eV
thetas = make_initial_thetas(nevents, theta)

output = run_MC(
    energies,
    thetas,
    earth,
    xs,
    seed=random_seed
)

In [ ]:
import matplotlib.pyplot as plt

pid_names = {
    11: r'$e$', 
    12: r'$\nu_{e}$', 
    13: r'$\mu$', 
    14: r'$\nu_{\mu}$',
    15: r'$\tau$',
    16: r'$\nu_{\tau}$'
}

bins  = np.logspace(12, 19, 75)
zeros = output['Eout']==0.

for pid in np.unique(output['PDG_Encoding']):
    particle_msk = np.logical_and(np.abs(output['PDG_Encoding'])==pid, ~zeros)
    name = pid_names[abs(pid)]ty
    plt.hist(output['Eout'][particle_msk], bins=bins, label=name,
            lw=2., histtype='step')
    
plt.legend(frameon=False, loc=2)
plt.loglog()
plt.xlabel(r'$E~\left(\rm{eV}\right)$')
plt.ylabel('Number')
plt.show()

## Example 2: Power law flux

`taurunner` also provides helper functions to propagate other spectral shapes, such as power laws or pre-loaded models of cosmogenic neutrinos.
Here, we are choosing to propagate an $E^{-2}$ spectrum over an entire hemisphere. 
There are some additional keyword arguments that allow the user to turn off tau energy losses (which are less important for more steeply inclined angles) or ignore secondary particles

In [ ]:
import numpy as np

from taurunner.main import run_MC
from taurunner.body.earth import construct_earth
from taurunner.cross_sections import CrossSections
from taurunner.utils import make_initial_e, make_initial_thetas

nevents = 5000 # number of events to simulate  
pid = 16
xs_model = "CSMS"
no_secondaries = True
random_seed = 925

np.random.seed(random_seed)

Earth = construct_earth(layers=[(4., 1.0)])
xs = CrossSections(xs_model)

# Sample power-law with index -2 between 1e6 GeV and 1e12 GeV
pl_exp = -2 # power law exponent
e_min = 1e15 # Minimum energy to sample in eV
e_max = 1e21 # Maximum energy to sample in eV
energies = make_initial_e(
    nevents,
    pl_exp, 
    e_min=e_max, 
    e_max=e_min
)

# Sample uniform in solid angle over hemisphere                    
th_min = 0 # Minimum nadir angle to sample from
th_max = 90 # Maximum nadir angle to sample from
thetas = make_initial_thetas(
    nevents, 
    (th_min, th_max), 
)

output = run_MC(
    energies, 
    thetas, 
    Earth, 
    xs,
    seed=random_seed,
    no_secondaries=no_secondaries
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

zeros = output['Eout']==0.
nutau_msk = np.logical_and(np.abs(output['PDG_Encoding'])==16, ~zeros)
plt.hist2d(
    output['Eini'][nutau_msk], 
    np.cos(np.radians(output['Theta'][nutau_msk])), 
    bins=[
        np.logspace(15., 21., 21), 
        np.linspace(0., 1., 21)
    ],
    norm=LogNorm()
)

plt.xscale('log')
plt.xlabel(r'$E_{\rm{initial}}~(\rm{eV})$')
plt.ylabel(r'$\cos(\theta)$')
plt.title("Initial Energy Distribution")
plt.show()

In [ ]:
from matplotlib.colors import LogNorm

bins  = np.logspace(12, 19, 75)
zeros = output['Eout']==0.
nutau_msk = np.logical_and(np.abs(output['PDG_Encoding'])==16, ~zeros)
plt.hist2d(
    output['Eout'][nutau_msk], 
    np.cos(np.radians(output['Theta'][nutau_msk])), 
    bins=[np.logspace(11, 18., 21), 
          np.linspace(0., 1., 21)],
    norm=LogNorm()
)

plt.xscale('log')
plt.xlabel(r'$E_{\rm{final}}~(\rm{eV})$')
plt.ylabel(r'$\cos(\theta)$')
plt.title("Final Energy Distribution")
plt.show()

Here, it is evident that the neutrinos that were propagated through more Earth (larger $\cos(\theta)$) exit the Earth at much lower energies than the neutrinos that were able to pass through skimming trajectories undergoing few, if any, interactions

We can also look at the one-dimensional energy spectra

In [ ]:
zeros = output['Eout']==0.

for pid in np.unique(output['PDG_Encoding']):
    particle_msk = np.logical_and(np.abs(output['PDG_Encoding'])==pid, ~zeros)
    name = pid_names[abs(pid)]
    plt.hist(output['Eout'][particle_msk], bins=bins, label=name,
             lw=2., histtype='step')
    
plt.legend(frameon=False, loc=1)
plt.loglog()
plt.xlabel(r'$E$ (eV)')
plt.ylabel('Number')
plt.show()

Notice how there are only tau neutrinos because we chose to not propagate any secondary particles. 

## Example 3: Propagating a custom flux through Earth

`taurunner` allows the user to supply CDFs in energy to be samlped from. Here we show an example of sampling from a GZK model. An example of how to create these CDFs is given in `make_flux_CDF.ipynb`

NOTE: If you downloaded the code using the versioins released on PyPI (i.e. you did not clone the github repository), then you will not have access to the default CDF, and you should instead make your own CDF using the example notebook linked above

In [ ]:
import numpy as np

import taurunner as tr
from taurunner.main import run_MC
from taurunner.body.earth import construct_earth
from taurunner.cross_sections import CrossSections
from taurunner.utils import make_initial_e, make_initial_thetas

nevents = 5000
pid = 16
xs_model = "CSMS"
random_seed = 925

Earth = construct_earth(layers=[(4., 1.0)])
xs = CrossSections(xs_model)

# Sample from pickled CDF
pkl_f = f'{tr.__path__[0]}/resources/ahlers2010_test.pkl' # Path to pickle file with CDF to sample from

np.random.seed(random_seed)

energies = make_initial_e(
    nevents,
    pkl_f, 
)
                         
# Sample uniform in solid angle over hemisphere   
th_min = 0 # Minimum nadir angle to sample from
th_max = 90 # Maximum nadir angle to sample from  
thetas = make_initial_thetas(
    nevents, 
    (th_min, th_max), 
)

output = run_MC(
    energies, 
    thetas, 
    Earth, 
    xs,
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

bins  = np.logspace(12, 19, 75)
zeros = output['Eout']==0.
nutau_msk = np.logical_and(np.abs(output['PDG_Encoding'])==16, ~zeros)
plt.hist2d(
    output['Eini'][nutau_msk], 
    np.cos(np.radians(output['Theta'][nutau_msk])), 
    bins=[np.logspace(12., 19., 27), 
          np.linspace(0., 1., 21)],
    norm=LogNorm()
)

plt.xscale('log')
plt.xlabel(r'$E_{\rm{initial}}$ (eV)')
plt.ylabel(r'$\cos(\theta)$')
plt.title("Initial Energy Distribution")
plt.show()

In [ ]:
from matplotlib.colors import LogNorm

bins  = np.logspace(12, 19, 75)
zeros = output['Eout']==0.
nutau_msk = np.logical_and(np.abs(output['PDG_Encoding'])==16, ~zeros)
plt.hist2d(
    output['Eout'][nutau_msk], 
    np.cos(np.radians(output['Theta'][nutau_msk])), 
    bins=[np.logspace(12., 19., 27), 
          np.linspace(0., 1., 21)],
    norm=LogNorm()
)

plt.xscale('log')
plt.xlabel(r'$E_{\rm{final}}$ (eV)')
plt.ylabel(r'$\cos(\theta)$')
plt.title("Final Energy Distribution")
plt.show()

## Example 4: Radial trajectories

TauRunner supports custom trajectories besides chords. TauRunner includes a radial trajectory which moves radially outward from the center of the body

In [ ]:
import numpy as np

from taurunner.main import run_MC
from taurunner.body.earth import construct_earth
from taurunner.cross_sections import CrossSections
from taurunner.utils import make_initial_e

nevents  = 5000
eini = 1e19
pid = 16
xs_model = "CSMS"

Earth = construct_earth(layers=[(4., 1.0)]) # Make Earth object with 4km water layer
xs = CrossSections(xs_model)
energies = make_initial_e(nevents, eini)
thetas = np.zeros(nevents)

output = run_MC(
    energies, 
    thetas,
    Earth, 
    xs, 
)

In [ ]:
import matplotlib.pyplot as plt

pid_names = {
    11: r'$e$', 
    12: r'$\nu_{e}$', 
    13: r'$\mu$', 
    14: r'$\nu_{\mu}$',
    15: r'$\tau$',
    16: r'$\nu_{\tau}$'
}

bins  = np.logspace(12, 19, 75)
zeros = output['Eout']==0.

for pid, name in pid_names.items():
    particle_msk = np.logical_and(np.abs(output['PDG_Encoding'])==pid, ~zeros)
    # name = pid_names[pid]
    plt.hist(
        output['Eout'][particle_msk],
        bins=bins,
        label=name,
        lw=2.,
        histtype='step'
    )
    
plt.legend(frameon=False, loc=1)
plt.loglog()
plt.xlabel(r'$E~\left(\rm{eV}\right)$')
plt.ylabel('Number')
plt.show()

## Example 5: Propagating through other bodies

In addition to propagating neutrinos through the Earth, you can also propagate neutrinos through the Sun or through any object given a density profile

In [ ]:
from taurunner.body import construct_sun
from taurunner.body import Body

### First, through the Sun

In [ ]:
import numpy as np
from taurunner.main import run_MC
from taurunner.body import construct_sun
from taurunner.cross_sections import CrossSections
from taurunner.utils import make_initial_e, make_initial_thetas, units

nevents = 5000
eini = 1e13 # the sun is opaque at high energies
theta = 10.0
pid = 16
xs_model = "dipole"
solar_model = "HZ_Sun" # Can also be "LZ_Sun"
random_seed = 925
                                   
xs = CrossSections(xs_model)
energies = make_initial_e(nevents, eini)
thetas = make_initial_thetas(nevents, theta)

sun = construct_sun(solar_model)

output = run_MC(
    energies, 
    thetas, 
    sun, 
    xs,
    seed=random_seed
)

In [ ]:
import matplotlib.pyplot as plt

bins  = np.logspace(1, 3.5, 26)
zeros = output['Eout']==0.

pid_names = {
    11: 'r$e$', 
    12: r'$\nu_{e}$', 
    13: r'$\mu$', 
    14: r'$\nu_{\mu}$',
    15: r'$\tau$',
    16: r'$\nu_{\tau}$'
}


for pid in reversed(range(12,17)):
    particle_msk = np.logical_and(np.abs(output['PDG_Encoding'])==pid, ~zeros)
    name = pid_names[pid]
    plt.hist(output['Eout'][particle_msk]/units.GeV, bins=bins, label=name,
            lw=2., histtype='step')
    
plt.legend(frameon=False, loc=1)
plt.loglog()
plt.xlabel(r'$E~\left(\rm{GeV}\right)$')
plt.ylabel('Number')
plt.show()

### And next, through a slab of constant density which we construct using the `Body` object and a radial trajectory

In [ ]:
import numpy as np

from taurunner.body import Body
from taurunner.main import run_MC
from taurunner.cross_sections import CrossSections
from taurunner.utils import make_initial_e, make_initial_thetas

nevents  = 5000
eini = 1e17
theta = 0
pid = 14
xs_model = "CSMS"

# Make body with density 3.14 g/cm^3 and radius 1000 km
body = Body(3.14, 1e3)

xs = CrossSections(xs_model)
energies = make_initial_e(nevents, eini)
thetas = make_initial_thetas(nevents, theta)

output   = run_MC(
    energies, 
    thetas, 
    body, 
    xs,
    flavor=pid
)

In [ ]:
import matplotlib.pyplot as plt

bins  = np.logspace(5, 19, 26)
zeros = output['Eout']==0.

pid_names = {
    11: 'r$e$', 
    12: r'$\nu_{e}$', 
    13: r'$\mu$', 
    14: r'$\nu_{\mu}$',
    15: r'$\tau$',
    16: r'$\nu_{\tau}$'
}

for pid in np.unique(output['PDG_Encoding']):
    particle_msk = np.logical_and(output['PDG_Encoding']==pid, ~zeros)
    name = pid_names[abs(pid)]
    plt.hist(output['Eout'][particle_msk], bins=bins, label=name,
            lw=2., histtype='step')
    
plt.plot()
plt.xlim(1e11,1e15)
plt.legend(frameon=False, loc=1)
plt.loglog()
plt.xlabel(r'$E$ (eV)')
plt.ylabel('Number')
plt.show()

### And next, through a layered slab

The constant density slab may be generalized to a slab of multiple layers. The densities in each layer may be positive scalars, unary functions which return positive scalars, or a potentially mixed list of such objects. In this example, we show how to accomplish this latter option.

In [ ]:
import numpy as np

from taurunner.body import Body
from taurunner.main import run_MC
from taurunner.cross_sections import CrossSections
from taurunner.utils import make_initial_e, make_initial_thetas

nevents  = 1000
eini = 1e15
theta = 0
pid = 16
xs_model = "CSMS"

# Make layered body with radius 1,000 km
def density_f(x):
    return x**-2/4
densities  = [4, density_f, 1, 0.4]
boundaries = [0.25, 0.3, 0.5, 1] # Right hand boundaries of the layers last boundary should always be 1
body = Body([(d, b) for d, b in zip(densities, boundaries)], 1e3)

xs = CrossSections(xs_model)
energies = make_initial_e(nevents, eini)
thetas = make_initial_thetas(nevents, theta)

output = run_MC(
    energies, 
    thetas, 
    body, 
    xs,
)

In [ ]:
import matplotlib.pyplot as plt

bins  = np.logspace(12.5, 19, 26)
zeros = output['Eout']==0.

pid_names = {
    11: 'r$e$', 
    12: r'$\nu_{e}$', 
    13: r'$\mu$', 
    14: r'$\nu_{\mu}$',
    15: r'$\tau$',
    16: r'$\nu_{\tau}$'
}


for pid in reversed(range(12,17)):
    particle_msk = np.logical_and(np.abs(output['PDG_Encoding'])==pid, ~zeros)
    name = pid_names[pid]
    plt.hist(output['Eout'][particle_msk], bins=bins, label=name,
            lw=2., histtype='step')
    
plt.legend(frameon=False, loc=1)
plt.loglog()
plt.xlabel(r'$E$ (eV)')
plt.ylabel('Number')
plt.show()

## Example 4: Running from the command line

As is described more thoroughly in the `README`, `taurunner` can also be called from the command line, an example of which is shown inline below

In [ ]:
!python ../taurunner/main.py -n 20 -t 0.0 -e 1e18